In [ ]:
#Importaciones
import pandas as pd
import kagglehub


from kagglehub import KaggleDatasetAdapter
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer

In [10]:
#Carga del dataset desde KaggleHub

file_path = "train.csv"

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mexwell/amazon-reviews-multi",
    file_path
)

#Revisión del dataset
print("Shape:", df.shape)
print("Columnas:", df.columns)
print("Primeras 5 filas: \n", df.head())

C:\Users\natal\AppData\Local\Temp\ipykernel_12924\3017153826.py:5: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Shape: (1200000, 9)
Columnas: Index(['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars',
       'review_body', 'review_title', 'language', 'product_category'],
      dtype='object')
Primeras 5 filas: 
    Unnamed: 0   review_id          product_id          reviewer_id  stars  \
0           0  de_0203609  product_de_0865382  reviewer_de_0267719      1   
1           1  de_0559494  product_de_0678997  reviewer_de_0783625      1   
2           2  de_0238777  product_de_0372235  reviewer_de_0911426      1   
3           3  de_0477884  product_de_0719501  reviewer_de_0836478      1   
4           4  de_0270868  product_de_0022613  reviewer_de_0736276      1   

                                         review_body  \
0     Armband ist leider nach 1 Jahr kaputt gegangen   
1                 In der Lieferung war nur Ein Akku!   
2  Ein Stern, weil gar keine geht nicht. Es hande...   
3  Dachte, das wären einfach etwas festere Binden...   
4  Meine Kinder haben kaum damit gespielt 

In [11]:
#preprocesamiento. ESTRUCTURA DE LAS REVIEWS

df["text"] = df["review_title"].fillna("") + " " + df["review_body"].fillna("")
df["labels"] = df["stars"] - 1
df = df[["text", "labels", "language"]]

In [12]:
#Conversión a hugging face dataset
dataset = Dataset.from_pandas(df)

#División del dataset train/test
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_val = dataset["train"].train_test_split(test_size=0.1, seed=42)

train_dataset = train_val["train"]
validation_dataset = train_val["test"]
test_dataset = dataset["test"]

In [13]:
#Tokenización
# NOTA: Si eligen otro modelo. esta parte se corrige dado que la tokenización no será la misma. Yo me encargo de ello
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
  return tokenizer(
      batch["text"],
      truncation=True,
      padding="max_length",
      max_length=180
  )

train_dataset = train_dataset.map(preprocess, batched=True)
validation_dataset = validation_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids","attention_mask", "labels"])
validation_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

c:\Users\natal\Documents\Projects\RN\RN-course project\NR-project-ARC\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\natal\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 120000/120000 [00:16<00:00, 7438.21 examples/s]
